In [ ]:
from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp

import pylib


In [ ]:
nbm.print_metadata()


In [ ]:
df = pylib.util.concat_dataframes_from_glob(
    "a=snapshot-comparisons-stats+*+ext=.csv"
)


In [ ]:
df


In [ ]:
all(df["quartet distance"] < 0.66)


In [ ]:
df["Generation"] = df["first rank"]
df["Population Size Estimate"] = df["population size estimate at first rank"]
df["Treatment"] = df["treatment"].str.replace("-", "\n")


def boxplot_popsize(data, x, y, hue):
    sns.boxplot(
        data=data,
        x=x,
        y=y,
        hue=hue,
        notch=True,
    )
    plt.yscale("log")


tp.tee(
    boxplot_popsize,
    data=df,
    x="Treatment",
    y="Population Size Estimate",
    hue="Generation",
)


In [ ]:
df["Generation A"] = df["first rank"]
df["Generation B"] = df["second rank"]


def plot_heatmap(data, x, y, hue, **kwargs):
    pivoted_data = (
        data[data[hue]]
        .pivot_table(index=x, columns=y, aggfunc="size", fill_value=0)
        .astype(int)
    )
    if len(pivoted_data) == 0:
        pivoted_data = data.pivot_table(
            index=x, columns=y, aggfunc="size", fill_value=0
        ).astype(int)
        pivoted_data[:] = 0

    sns.heatmap(
        pivoted_data,
        xticklabels=True,
        yticklabels=True,
        vmin=0,
        vmax=10,
        mask=np.tril(np.ones_like(pivoted_data.values, dtype=bool), -1),
        annot=True,
        fmt="d",
        cmap="viridis",
        cbar=False,
        **kwargs
    )


def facet_heatmap(data, x, y, hue):
    g = sns.FacetGrid(data, col="treatment", col_wrap=2, height=4, aspect=1)
    g.map_dataframe(plot_heatmap, x=x, y=y, hue=hue)


tp.tee(
    facet_heatmap,
    data=df,
    x="Generation B",
    y="Generation A",
    hue="Mann-Whitney significant at alpha = 0.01",
)


In [ ]:
tp.tee(
    facet_heatmap,
    data=df,
    x="Generation B",
    y="Generation A",
    hue="nonoverlapping CI",
)
